<a href="https://colab.research.google.com/github/abdelrahman-tarek-0/Nlu-tasks-2025/blob/main/task-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers datasets faiss-cpu scikit-learn pandas rank_bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" an

In [2]:
from datasets import load_dataset
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, util
import torch
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset")["train"]
dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


README.md:   0%|          | 0.00/233 [00:00<?, ?B/s]

medDataset_processed.csv:   0%|          | 0.00/22.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16407 [00:00<?, ? examples/s]

Dataset({
    features: ['qtype', 'Question', 'Answer'],
    num_rows: 16407
})

In [4]:
questions = dataset["Question"]
answers = dataset["Answer"]

In [5]:
model = SentenceTransformer("all-MiniLM-L6-v2")
answer_embeddings = model.encode(answers, convert_to_tensor=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
query = "Who is at risk for Lymphocytic Choriomeningitis (LCM) ?"
query_embedding = model.encode(query, convert_to_tensor=True)
cosine_scores = util.pytorch_cos_sim(query_embedding, answer_embeddings)[0]

In [7]:
top_k = 5
top_results = torch.topk(cosine_scores, k=top_k)

print(f"\n🔍 Query: {query}\n")
print("📚 Top 5 Relevant Answers:\n")
for score, idx in zip(top_results[0], top_results[1]):
    print(f"[Score: {score:.4f}]")
    print(f"Answer: {answers[idx]}")
    print(f"Matched Question: {questions[idx]}")
    print("-" * 90)


🔍 Query: Who is at risk for Lymphocytic Choriomeningitis (LCM) ?

📚 Top 5 Relevant Answers:

[Score: 0.5409]
Answer: LCH is a rare condition, although its prevalence is unknown.
Matched Question: How many people are affected by lissencephaly with cerebellar hypoplasia ?
------------------------------------------------------------------------------------------
[Score: 0.4964]
Answer: It is estimated that 500 to 1,000 people worldwide have chorea-acanthocytosis.
Matched Question: How many people are affected by chorea-acanthocytosis ?
------------------------------------------------------------------------------------------
[Score: 0.4948]
Answer: Langerhans cell histiocytosis (LCH) is a disorder that primarily affects children, but is also found in adults of all ages. People with LCH produce too many Langerhans cells or histiocytes, a form of white blood cell found in healthy people that is supposed to protect the body from infection. In people with LCH, these cells multiply excessivel